# Instala bibliotecas necessárias e importa módulos essenciais.



In [ ]:
!pip install -q unsloth[colab-new] faiss-cpu sentence-transformers datasets accelerate trl peft transformers
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

In [ ]:
import json
import re
import os
import unicodedata
import faiss
import numpy as np
import torch
import random
import pandas as pd
import sqlite3

from unsloth import FastLanguageModel, is_bfloat16_supported
from transformers import pipeline
from transformers import TrainingArguments
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
from datasets import Dataset
from trl import SFTTrainer
from dotenv import load_dotenv
from huggingface_hub import login
from datetime import date, timedelta

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Download e exploração inicial dos dados

In [ ]:
!git clone https://github.com/pubmedqa/pubmedqa.git

file_path = 'pubmedqa/data/ori_pqal.json'

with open(file_path, 'r') as f:
    data = json.load(f)

sample_key = list(data.keys())[0]
print(f"\nCampos disponíveis: {list(data[sample_key].keys())}\n")

print("=" * 60)
print("Exploração de dados - PubMedQA")
print("=" * 60)

for i, key in enumerate(list(data.keys())[:3]):
    item = data[key]

    print(f"\nExemplo {i+1} | ID: {key}")
    print("-" * 60)
    print(f"Question: {item.get('QUESTION', 'N/A')}")

    context = " ".join(item.get('CONTEXTS', []))
    print(f"Context: {context[:300]}...")

    print(f"Labels: {item.get('LABELS', 'N/A')}")
    print(f"Decision: {item.get('final_decision', 'N/A')}")
    print(f"Answer: {item.get('LONG_ANSWER', 'N/A')[:200]}...")
    print(f"Meshes: {item.get('MESHES', 'N/A')}")
    print(f"Year: {item.get('YEAR', 'N/A')}")
    print(f"Reasoning required pred: {item.get('reasoning_required_pred', 'N/A')}")
    print(f"Reasoning free pred: {item.get('reasoning_free_pred', 'N/A')}")

print(f"\n\nTotal de registros: {len(data)}")

# Pré-processamento - Limpeza, Normalização e Anonimização dos Textos

In [ ]:
MAP_DECISION = {
  "yes": "SIM",
  "no": "NÃO",
  "maybe": "TALVEZ"
}


def anonymize_text(text):
    """Remove dados sensíveis (LGPD/HIPAA compliance)"""
    if not isinstance(text, str):
        return ""

    text = re.sub(r'(Dr\.|Dra\.|Doctor|Prof\.|MD)\s+[A-Z][a-z]+(\s+[A-Z][a-z]+)?', '[NOME]', text)
    text = re.sub(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', '[EMAIL]', text)
    locations = r'(Israel|Denmark|Chile|Texas|France|United Kingdom|UK|USA|Pakistan|Karachi|Jordan|Japan|Australia|North Carolina|Washington)'
    text = re.sub(locations, '[LOCAL]', text, flags=re.IGNORECASE)
    text = re.sub(r'\b\d{6,}\b', '[ID]', text)
    text = re.sub(r'\b(19|20)\d{2}\b', '[ANO]', text)
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '[URL]', text)
    return text

def clean_text(text):
  if not text:
    return ""
  text = unicodedata.normalize("NFKC", text)
  text = re.sub(r"\s+", " ", text).strip()

  text = anonymize_text(text)

  return text


processed_docs = []

for item in data.values():
  question = clean_text(item.get("QUESTION"))
  context = clean_text(" ".join(item.get("CONTEXTS", [])))
  answer = clean_text(item.get("LONG_ANSWER", ""))
  decision = MAP_DECISION.get(item.get("final_decision", ""), "")


  if question and context:
    processed_docs.append({
      "question": question,
      "context": context,
      "answer": answer,
      "decision": decision,
      "year": item.get("YEAR")
    })


print(len(processed_docs))

# Preparando conjunto de dados em português para treino de tradução

In [ ]:
!git clone https://github.com/diegosdomingos/tech-challenge-3.git

file_path = 'tech-challenge-3/data/language_alignment_pt.jsonl'

dataset = load_dataset(
    "json",
    data_files="/content/drive/MyDrive/rag/language_alignment_pt.json",  #Alterar quando o dataset já estiver na main
    split="train"
)

print(dataset.column_names)

def messages_to_text(example):
    text = ""
    for msg in example["messages"]:
        if msg["role"] == "system":
            text += f"<<SYS>>\n{msg['content']}\n<</SYS>>\n\n"
        elif msg["role"] == "user":
            text += f"[INST] {msg['content']} [/INST]\n"
        elif msg["role"] == "assistant":
            text += msg["content"]
    return {"text": text}

dataset = dataset.map(
    messages_to_text,
    batched=False,
    remove_columns=["messages"]
)

print(dataset.column_names)
print(dataset[0])

# Estruturação de Documentos para RAG (Recuperação + Geração)
## - Cria textos formatados unificando questão, contexto e resposta para busca semântica.

In [ ]:
rag_documents = []

for d in processed_docs:
  text = f"""
  Pergunta científica:
  {d['question']}


  Evidência:
  {d['context']}


  Conclusão:
  {d['answer']}
  """
  rag_documents.append(text.strip())


len(rag_documents)

# Geração de Embeddings e Construção de Índice FAISS

In [ ]:
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = embedder.encode(rag_documents, show_progress_bar=True)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

# Create the directory if it doesn't exist
os.makedirs('/content/drive/MyDrive/rag', exist_ok=True)

faiss.write_index(index, "/content/drive/MyDrive/rag/medical_index.faiss")

with open('/content/drive/MyDrive/rag/medical_docs.json', 'w') as f:
  json.dump(rag_documents, f)

# Configuração do Modelo para Treinamento com LoRA
## - Carrega modelo base e aplica adaptação LoRA para reduzir custo de treino.

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
    device_map="auto"
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0.05,
)

# Treinamento Supervisionado do Assistente Médico em Português

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    learning_rate=2e-4,
    logging_steps=5,
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=training_args,
    dataset_text_field="text",
    max_seq_length=512,
    packing=False
)

trainer.train()

# Autenticação e Upload do Modelo Treinado no HuggingFace

In [ ]:

# Carregar env
ENV_PATH = "/content/drive/MyDrive/token-hf/env"
load_dotenv(ENV_PATH)

HF_TOKEN = os.getenv("HF_TOKEN")
login(token=HF_TOKEN)

HF_REPO = f"{HF_USER_REPO}/assistente-medico-lora"

model.push_to_hub(HF_REPO)
tokenizer.push_to_hub(HF_REPO)

# Configuração da Pipeline de Geração + Prompt do Assistente Médico

In [ ]:
SYSTEM_PROMPT = """
Você é um assistente médico virtual.
Responda sempre em português, com linguagem clara, empática e baseada em evidências científicas.


Regras:
- Não invente informações.
- Se não houver evidência suficiente, diga isso explicitamente.
- Não prescreva medicamentos nem indique tratamentos específicos.
- Não indique remédios ou tratamentos.
- Quando perguntarem por algum remédio, você deve responder: Não estou autorizado a prescrever medicamentos, por favor, consulte um médico. <FIM>
- Sempre cite a fonte da informação científica

Responda de forma objetiva e finalize sempre a primeira resposta objetiva com o texto: <FIM>
"""

with open('/content/drive/MyDrive/rag/medical_docs.json') as f:
  docs = json.load(f)

index = faiss.read_index('/content/drive/MyDrive/rag/medical_index.faiss')


llm = pipeline(
  "text-generation",
  model=model,
  tokenizer=tokenizer,
  max_new_tokens=512,
  temperature=0.0,
  do_sample=False,
  repetition_penalty=1.1,
  return_full_text=False,
  eos_token_id=tokenizer.eos_token_id
  )

def retrieve_context(question, k=3):
  q_emb = embedder.encode([question])
  _, idx = index.search(q_emb, k)
  return "\n\n".join([docs[i] for i in idx[0]])

def medical_chat(question):

  # Adiciona <FIM> ao final da string `question` se não estiver presente
  if "<FIM>" not in question:
    question += " <FIM>"

  context = retrieve_context(question)

  prompt = f"""
{SYSTEM_PROMPT}


Contexto científico relevante:
{context}


Pergunta: {question}
Resposta:
"""
  output = llm(prompt)[0]["generated_text"]

  # 🔒 corta tudo depois do token de fim
  output = output.split("<FIM>")[0]

  return output.strip()

# Criação de dataset de prontuário (Fictício) com SQLite

In [ ]:
DB_PATH = "prontuarios.db"

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS pacientes (
    patient_id TEXT PRIMARY KEY,
    nome TEXT,
    data_nascimento TEXT,
    idade INTEGER,
    sexo TEXT,
    alergias TEXT,
    comorbidades TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS atendimentos (
    atendimento_id INTEGER PRIMARY KEY AUTOINCREMENT,
    patient_id TEXT,
    data_atendimento TEXT,
    queixa_principal TEXT,
    anamnese TEXT,
    diagnostico TEXT,
    conduta TEXT,
    tratamentos_em_andamento TEXT,
    exames_solicitados TEXT,
    observacoes TEXT,
    FOREIGN KEY(patient_id) REFERENCES pacientes(patient_id)
)
""")

conn.commit()
conn.close()

# Populando os dados fictícios

In [ ]:
nomes = [
    "Ana Paula Souza", "Ana Carolina Lima", "Bruno Silva", "Carlos Eduardo Rocha",
    "Daniela Martins", "Eduardo Nogueira", "Fernanda Alves", "Gabriel Pacheco",
    "Helena Ribeiro", "Igor Farias", "Juliana Torres", "Lucas Fernandes",
    "Mariana Araujo", "Natalia Pacheco", "Otavio Nunes", "Paula Guedes",
    "Rafael Moreira", "Sabrina Lopes", "Thiago Barros", "Vanessa Farias",
    "William Teixeira", "Ana Beatriz Costa"
]

diagnosticos = [
    "Hipertensão arterial sistêmica",
    "Diabetes mellitus tipo 2",
    "Asma brônquica",
    "Infecção do trato urinário",
    "Pneumonia adquirida na comunidade",
    "Transtorno de ansiedade generalizada",
    "Gastrite crônica",
    "Enxaqueca crônica"
]

alergias_lista = [
    "Dipirona", "Penicilina", "Sulfa", "Nenhuma conhecida"
]

comorbidades_lista = [
    "Hipertensão", "Diabetes", "Dislipidemia", "Obesidade", "Nenhuma"
]

def gerar_data_nascimento(idade):
    hoje = date.today()
    return hoje - timedelta(days=idade * 365)

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

for i, nome in enumerate(nomes, start=1):
    idade = random.randint(18, 85)
    patient_id = f"PAT{i:04d}"

    cursor.execute("""
        INSERT OR IGNORE INTO pacientes
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (
        patient_id,
        nome,
        gerar_data_nascimento(idade).isoformat(),
        idade,
        random.choice(["F", "M"]),
        random.choice(alergias_lista),
        random.choice(comorbidades_lista)
    ))

    for _ in range(random.randint(1, 4)):
        cursor.execute("""
            INSERT INTO atendimentos (
                patient_id,
                data_atendimento,
                queixa_principal,
                anamnese,
                diagnostico,
                conduta,
                tratamentos_em_andamento,
                exames_solicitados,
                observacoes
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            patient_id,
            (date.today() - timedelta(days=random.randint(1, 1200))).isoformat(),
            "Dor, mal-estar e sintomas gerais",
            "Paciente relata início dos sintomas há alguns dias, sem fatores agravantes claros.",
            random.choice(diagnosticos),
            "Conduta expectante e acompanhamento ambulatorial",
            "Uso contínuo de medicação conforme prescrição",
            "Hemograma completo, glicemia, PCR",
            "Paciente orientado quanto aos sinais de alarme"
        ))

conn.commit()
conn.close()

# Regras para consultar prontuários

In [ ]:
def buscar_pacientes_por_nome(nome_parcial: str):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    cursor.execute("""
        SELECT patient_id, nome, idade
        FROM pacientes
        WHERE LOWER(nome) LIKE LOWER(?)
    """, (f"%{nome_parcial}%",))

    resultados = cursor.fetchall()
    conn.close()
    return resultados


def carregar_prontuario(patient_id: str):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    cursor.execute("""
        SELECT patient_id, nome, data_nascimento, idade, sexo, alergias, comorbidades
        FROM pacientes
        WHERE patient_id = ?
    """, (patient_id,))
    paciente = cursor.fetchone()

    cursor.execute("""
        SELECT data_atendimento, queixa_principal, anamnese, diagnostico,
               conduta, tratamentos_em_andamento, exames_solicitados, observacoes
        FROM atendimentos
        WHERE patient_id = ?
        ORDER BY data_atendimento DESC
    """, (patient_id,))
    atendimentos = cursor.fetchall()

    conn.close()

    if not paciente:
        return "Paciente não encontrado."

    texto = f"""
PACIENTE
ID: {paciente[0]}
Nome: {paciente[1]}
Data de nascimento: {paciente[2]}
Idade: {paciente[3]}
Sexo: {paciente[4]}
Alergias: {paciente[5]}
Comorbidades: {paciente[6]}

ATENDIMENTOS:
"""

    for a in atendimentos:
        texto += f"""
Data: {a[0]}
Queixa principal: {a[1]}
Anamnese: {a[2]}
Diagnóstico: {a[3]}
Conduta: {a[4]}
Tratamentos em andamento: {a[5]}
Exames solicitados: {a[6]}
Observações: {a[7]}
---
"""

    return texto


# LLM para entender a pergunta do prontuário (Utilização da mesma llm LLaMA)

In [ ]:
def extrair_intencao_e_nome(pergunta: str):
    prompt = f"""
Extraia informações estruturadas da pergunta abaixo.

Pergunta:
"{pergunta}"

Responda APENAS em JSON, sem nenhum texto extra:

{{
  "intencao": "consultar_prontuario" | "outra",
  "nome_paciente": string | null
}}

Responda de forma objetiva e finalize sempre a primeira resposta objetiva com o texto: <FIM>

"""
    resposta = medical_chat(prompt)

    try:
        json_str = re.search(r"\{.*\}", resposta, re.DOTALL).group()
        return json.loads(json_str)
    except:
        return {"intencao": "outra", "nome_paciente": None}


# Router (Consulta prontuário ou Dados de pubmedqa?)

In [ ]:
estado = {
    "aguardando_escolha": False,
    "opcoes": []
}

def router_chat(pergunta: str):

    # 1️⃣ Se o usuário precisa escolher um paciente
    if estado["aguardando_escolha"]:
        escolha = pergunta.strip().lower()

        for pid, nome, idade in estado["opcoes"]:
            if escolha == pid.lower() or escolha == nome.lower():
                estado["aguardando_escolha"] = False
                estado["opcoes"] = []
                prontuario = carregar_prontuario(pid)
                return medical_chat(prontuario)

        return medical_chat(
            "Ainda existem mais de um paciente possível. "
            "Pode informar o ID ou o nome completo?"
        )

    # 2️⃣ Entender a pergunta
    parsed = extrair_intencao_e_nome(pergunta)

    if parsed["intencao"] != "consultar_prontuario":
        return medical_chat(pergunta)

    nome = parsed["nome_paciente"]

    if not nome:
        return medical_chat(
            "Claro 🙂 Qual é o nome do paciente que você deseja consultar?"
        )

    pacientes = buscar_pacientes_por_nome(nome)

    if len(pacientes) == 0:
        return medical_chat(
            "Não encontrei nenhum paciente com esse nome. "
            "Você pode informar o nome completo?"
        )

    if len(pacientes) > 1:
        estado["aguardando_escolha"] = True
        estado["opcoes"] = pacientes

        lista = "\n".join(
            [f"- {nome} (ID: {pid}, Idade: {idade})"
             for pid, nome, idade in pacientes]
        )

        return medical_chat(
            "Encontrei mais de um paciente com esse nome:\n\n"
            f"{lista}\n\n"
            "Qual deles você deseja consultar?"
        )

    prontuario = carregar_prontuario(pacientes[0][0])
    return medical_chat(prontuario)


# Testes de Consulta ao Assistente Médico com Exemplos


In [ ]:
# Teste normal de consulta
question = "O que a literatura indica sobre o uso de aspirina em prevenção primária?"
print(f"Resposta 1 (normal): {medical_chat(question)}")

# Testa restrição
question = "Qual medicamento é eficaz para pedra nos rins?"
print(f"Resposta 2 (restrição): {medical_chat(question)}")

# CHAT Assistente

In [ ]:
print("Assistente Médico iniciado")
print("Digite 'sair' para encerrar\n")

while True:
    user_input = input("Você: ").strip()

    if user_input.lower() in ["sair", "exit", "quit"]:
        print("Assistente: Sessão encerrada.")
        break

    resposta = router_chat(user_input)
    print(f"Assistente: {resposta}\n")
